In [ ]:
#Allows importing functions from other notebooks
import notebook_finder
from imp import reload

In [ ]:
#Necessary to keep changing functions from other Notebook fresh
import notebook_of_functions as kung
reload(kung)
import notebook_of_functions as kung
reload(kung)

In [ ]:
#Grabs an Excel-style rectangualr range from Google Sheet
import pipulate as gs
import pandas as pd
tab = gs.name('Zen of Pipulate').sheet1
cl, df = gs.pipulate(tab, rows=(1, 20), cols=('a', 'b'))

In [ ]:
import this #copy output into column A

In [ ]:
#Changes pandas DataFrame then pushes changes to Google Sheet
df['B'] = kung.foo()
gs.populate(tab, cl, df)